# Sentiment Analysis of Italian Restaurant Reviews

The project aims to analyze customer sentiment towards Italian restaurants in France and Italy. The analysis is based on textual reviews collected from two different data sources: OpenTable and Yelp.

By employing sentiment analysis, we aim to uncover trends and compare how Italian cuisine and the general quality of the related businesses are perceived across the two regions.

## Yelp Fusion API

In [ ]:
import requests

# Set up API credentials
API_KEY = "YELP_FUSION_API_KEY"
API_URL = "https://api.yelp.com/v3/businesses/search"

# Define the search parameters
headers = {"Authorization": f"Bearer {API_KEY}"}
params = {
    "term": "",
    "location": "Italy",
    "categories": "restaurants",
    "limit": 10,  # Number of results to fetch
}

# Make the request
response = requests.get(API_URL, headers=headers, params=params)

# Parse the response
if response.status_code == 200:
    data = response.json()
    for business in data.get("businesses", []):
        print(f"Name: {business['name']}")
        print(f"Rating: {business['rating']}")
        print(f"Address: {', '.join(business['location']['display_address'])}")
        print(f"Phone: {business.get('phone', 'N/A')}")
        print("-" * 40)
else:
    print(f"Error: {response.status_code} - {response.text}")


Name: 44 Osteria Quarantaquattro
Rating: 4.5
Address: Via Aureliana 42, 00187 Rome, Italy
Phone: +390642013318
----------------------------------------
Name: Roscioli
Rating: 4.3
Address: Via dei Giubbonari 21, 00186 Rome, Italy
Phone: +39066875287
----------------------------------------
Name: Cantina e Cucina
Rating: 4.5
Address: Via del Governo Vecchio 87, 00186 Rome, Italy
Phone: +39066892574
----------------------------------------
Name: Hostaria La Botticella
Rating: 4.5
Address: Vicolo del Leopardo 39A, 00153 Rome, Italy
Phone: +39065814738
----------------------------------------
Name: Il Chianti
Rating: 4.3
Address: Via del Lavatore 81, 00187 Rome, Italy
Phone: +39066787550
----------------------------------------
Name: Ai Tre Scalini
Rating: 4.4
Address: Via Panisperna 251, 00184 Rome, Italy
Phone: +390648907495
----------------------------------------
Name: La Tavernetta 48
Rating: 4.4
Address: Via degli Spagnoli 48, 00186 Rome, Italy
Phone: +390668192591
-------------------

## OpenTable

In [2]:
%pip install selenium --quiet

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time


def scrape_opentable_reviews(url, max_reviews=10):
    """
    Scrapes reviews from an OpenTable restaurant page.

    Args:
    - url (str): The URL of the restaurant's OpenTable page.
    - max_reviews (int): The maximum number of reviews to scrape.

    Returns:
    - list of dict: A list of reviews with the review text and date.
    """
    # Initialize WebDriver
    driver = webdriver.Chrome()
    driver.get(url)
    time.sleep(5)

    reviews = []

    try:
        while len(reviews) < max_reviews:
            # Extract review elements
            review_elements = driver.find_elements(By.CLASS_NAME, 'afkKaa-4T28-')

            for review_element in review_elements:
                try:
                    review_text = review_element.find_element(By.CLASS_NAME, '_6rFG6U7PA6M-').text
                    review_date = review_element.find_element(By.CLASS_NAME, 'iLkEeQbexGs-').text
                    reviews.append({'review_text': review_text, 'review_date': review_date})

                    # Stop if we reach the maximum reviews
                    if len(reviews) >= max_reviews:
                        break
                except Exception as e:
                    print(f"Error extracting review: {e}")
                    continue

            # Attempt to click "Load More Reviews" if it exists
            try:
                load_more_button = driver.find_element(By.CSS_SELECTOR, '.ot-load-more-reviews')
                load_more_button.click()
                time.sleep(3)  # Wait for reviews to load
            except:
                print("No more reviews to load.")
                break

    except Exception as e:
        print(f"Error during scraping: {e}")
    finally:
        driver.quit()

    return reviews

In [11]:
opentable_url = ("https://www.opentable.com/r/sacre-frenchy-paris?corrid=400c4ade-c1da-4fb9-88a7-a165c57bab08&avt"
                "=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&p=2&sd=2024-11-22T19%3A00%3A00")
scraped_reviews = scrape_opentable_reviews(opentable_url, max_reviews=20)

for idx, review in enumerate(scraped_reviews, 1):
  print(f"Review {idx}:")
  print(f"Date: {review['review_date']}")
  print(f"Text: {review['review_text']}\n")

No more reviews to load.
Review 1:
Date: Dined 6 days ago
Text: Excellente soirée au sacré Frenchy ! Nous avons pu déguster des spécialités françaises. Service très agréable et des cocktails excellents.
Je reviendrai avec plaisir 🙃

Review 2:
Date: Dined on November 29, 2024
Text: We had a lovely meal, felt very at ease as all staff spoke very good English, the food was lovely, the drinks were great, all round a great place to visit x

Review 3:
Date: Dined on November 28, 2024
Text: Sacré Frenchy was a great place to go to for our American family of four with twin 16 year old daughters. Everyone enjoyed their meals and the staff was really accommodating considering we can’t speak French. Our girls can sometimes be picky eaters when we travel abroad and this was a really good experience as we were all able to find something to eat without having to sacrifice a good dining experience for my wife and I.
Read more

Review 4:
Date: Dined on November 26, 2024
Text: The duck is delicious!!! 